In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [6]:
import csv
import calendar
import datetime

# Clean Taxi Data
def parserTaxi(id, data):
    if id == 0:
        data.next()
    for i in csv.reader(data):
        tpep_pickup_datetime, tpep_dropoff_datetime, pickup_longitude, \
        pickup_latitude, dropoff_longitude, dropoff_latitude, Date, Weekday, HourofDay = i[1], i[2], i[5], i[6], i[9], i[10], \
        datetime.datetime.strptime(i[1], "%Y-%m-%d %H:%M:%S").date(), \
        calendar.day_name[datetime.datetime.strptime(i[1], "%Y-%m-%d %H:%M:%S").weekday()], \
        datetime.datetime.strptime(i[1], "%Y-%m-%d %H:%M:%S").hour
        # rows required for the taxi data
        yield tpep_pickup_datetime, tpep_dropoff_datetime, pickup_longitude, \
        pickup_latitude, dropoff_longitude, dropoff_latitude, Date, Weekday, HourofDay

In [5]:
for i in range(1, 4):
    taxi = sc.textFile('Data/top20-{0}.csv'.format(i), use_unicode=False).cache()
    taxiRDD = taxi.mapPartitionsWithIndex(parserTaxi)
    df = sqlContext.createDataFrame(taxiRDD, ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'Date', 'Weekday', 'HourofDay'])
    agg = df.select(['Date', 'HourofDay']).groupBy('Date').count()
    agg.coalesce(1).write.csv('Data/top20-{0}_AGG'.format(i), header=True)

21

In [18]:
taxi = sc.textFile('Data/top20-1.csv', use_unicode=False).cache()
taxiRDD = taxi.mapPartitionsWithIndex(parserTaxi)
df = sqlContext.createDataFrame(taxiRDD, ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'Date', 'Weekday', 'HourofDay'])
agg = df.select(['Date', 'HourofDay']).groupBy(['Date', 'HourofDay']).count()

In [12]:
agg2 = df.groupby(['Date', 'HourofDay'])

In [20]:
agg.take(3)

[Row(Date=datetime.date(2016, 6, 9), HourofDay=23, count=10),
 Row(Date=datetime.date(2016, 6, 9), HourofDay=22, count=6),
 Row(Date=datetime.date(2016, 6, 9), HourofDay=21, count=19)]

In [14]:
agg2.take(3)

AttributeError: 'GroupedData' object has no attribute 'take'

In [15]:
df.take(3)

[Row(tpep_pickup_datetime=u'2016-06-09 21:06:36', tpep_dropoff_datetime=u'2016-06-09 21:13:08', pickup_longitude=u'-73.983360290527344', pickup_latitude=u'40.760936737060547', dropoff_longitude=u'-73.977462768554688', dropoff_latitude=u'40.753978729248047', Date=datetime.date(2016, 6, 9), Weekday=u'Thursday', HourofDay=21),
 Row(tpep_pickup_datetime=u'2016-06-09 21:06:36', tpep_dropoff_datetime=u'2016-06-09 21:35:11', pickup_longitude=u'-73.981719970703125', pickup_latitude=u'40.736667633056641', dropoff_longitude=u'-73.981636047363281', dropoff_latitude=u'40.670242309570313', Date=datetime.date(2016, 6, 9), Weekday=u'Thursday', HourofDay=21),
 Row(tpep_pickup_datetime=u'2016-06-09 21:06:36', tpep_dropoff_datetime=u'2016-06-09 21:13:10', pickup_longitude=u'-73.994316101074219', pickup_latitude=u'40.751071929931641', dropoff_longitude=u'-74.004234313964844', dropoff_latitude=u'40.742168426513672', Date=datetime.date(2016, 6, 9), Weekday=u'Thursday', HourofDay=21)]